In [2]:
from datasets import load_dataset
import pandas as pd
from openai import OpenAI

/Users/simonwallace/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [120]:
client = OpenAI()

In [9]:
dataset = load_dataset("refugee-law-lab/canadian-legal-data", "FC", split="train")
df = pd.DataFrame(dataset)

In [ ]:
df = df[df['citation'].str[:4] == '2024']
df = df[df['language'] == 'en']


In [53]:
def extract_file(text):
    paras = text.split('\n')
    for idx,para in enumerate(paras):
        if 'file' in para.lower():
            return paras[idx+1]

In [111]:
df['files'] = df['unofficial_text'].apply(extract_file)
df = df[(df['files'].str[0] == 'I')]

/var/folders/54/pnyqyxcd4013kndyzbprlnxh0000gn/T/ipykernel_79573/2476011910.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['files'] = df['unofficial_text'].apply(extract_file)


In [112]:
def extract_judgement(text):
    paras = text.split('\n')
    for idx,para in enumerate(paras):
        if 'THIS C' in para:
            if ('granted' in para.lower()) or ('allowed' in para.lower()):
                return 'granted'
            if 'dismissed' in para.lower():
                return 'dismissed'
            if ('granted' in paras[idx+1].lower()) or ('allowed' in paras[idx+1].lower()):
                return 'granted'
            if 'dismissed' in paras[idx+1].lower():
                return 'dismissed'
            else:
                return 'other'
        

df['judgement'] = df['unofficial_text'].apply(extract_judgement)

In [113]:
df['judgement'].value_counts()

judgement
dismissed    641
granted      430
other         38
Name: count, dtype: int64

In [117]:
df = df[df['judgement'] == 'granted'].reset_index(drop=True)

In [127]:
sample = df.sample(10)
sample.reset_index(drop=True, inplace=True)

In [122]:
with open("prompt.txt", "r") as f:
    prompt = f.read()

In [128]:
for idx,row in sample.iterrows():
    if idx == 0:
        with open("manual v.0.txt", "r") as f:
            manual = f.read()
    else:
        with open(f"versions/manual v.{idx}.txt", "r") as f:
            manual = f.read()
    
    new_prompt = prompt.replace("{manual}", manual)
    new_prompt = new_prompt.replace("{case}", row['unofficial_text'])
    
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": new_prompt,
            }
        ],
        model="gpt-4o",
    )
    
    with open(f"versions/manual v.{idx+1}.txt", "w") as f:
        f.write(chat_completion.choices[0].message.content)